# Exportación de Visualizaciones - Análisis Demográfico

Este notebook exporta las visualizaciones del análisis demográfico a formatos estáticos (PNG) e interactivos (HTML).

## Requisitos Previos
1. Tener el notebook `03_Analisis_BigQuery_Demografia.ipynb` ejecutado previamente
2. Tener instalado kaleido para exportar a PNG: `pip install -U kaleido`
3. Tener los permisos necesarios en la carpeta de destino

## Funcionamiento
El notebook:
1. Carga los datos del análisis previo
2. Genera las visualizaciones
3. Las guarda en formato PNG y HTML

In [5]:
# Importar bibliotecas necesarias
import os
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Verificar dependencias necesarias
try:
    import kaleido
except ImportError:
    print("⚠️ kaleido no está instalado. Instalando...")
    #!pip install -U kaleido
    import kaleido

# Configurar directorios usando Path para mejor manejo de rutas
BASE_DIR = Path('e:/repos/ds_portfolio')
NOTEBOOKS_DIR = BASE_DIR / 'notebooks'
DATA_DIR = NOTEBOOKS_DIR / 'data'  # Aquí están los archivos nombres_demografia.*
VIZ_DIR = NOTEBOOKS_DIR / 'visualizaciones'  # Para guardar las visualizaciones

# Crear directorios si no existen
DATA_DIR.mkdir(parents=True, exist_ok=True)
VIZ_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Configuración inicial completada:")
print(f"📂 Directorio de datos: {DATA_DIR}")
print(f"📊 Directorio de visualizaciones: {VIZ_DIR}")

# Verificar que los archivos de datos existen
data_files = {
    'csv': DATA_DIR / 'nombres_demografia.csv',
    'pkl': DATA_DIR / 'nombres_demografia.pkl'
}

print("🔍 Verificando archivos y directorios:")
print("\n📂 Archivos de datos:")
for fmt, path in data_files.items():
    if path.exists():
        size = path.stat().st_size / 1024  # tamaño en KB
        print(f"✅ {path.name} ({size:.1f} KB)")
    else:
        print(f"❌ {path.name} no encontrado")

print("\n📊 Directorio de visualizaciones:")
VIZ_DIR.mkdir(parents=True, exist_ok=True)
print(f"✅ {VIZ_DIR} {'existe' if VIZ_DIR.exists() else 'creado'}")

✅ Configuración inicial completada:
📂 Directorio de datos: e:\repos\ds_portfolio\notebooks\data
📊 Directorio de visualizaciones: e:\repos\ds_portfolio\notebooks\visualizaciones
🔍 Verificando archivos y directorios:

📂 Archivos de datos:
✅ nombres_demografia.csv (2.6 KB)
✅ nombres_demografia.pkl (0.0 KB)

📊 Directorio de visualizaciones:
✅ e:\repos\ds_portfolio\notebooks\visualizaciones existe


## 1. Cargar el DataFrame desde archivo temporal

Para evitar reejecutar las consultas de BigQuery, primero guardaremos el DataFrame en un archivo temporal CSV.

## 1. Importar datos del notebook principal

Para acceder a los datos del análisis demográfico, necesitamos:
1. Copiar el DataFrame del notebook principal
2. O cargarlo desde una variable compartida

Método 1 - Copiar el DataFrame:
```python
# En el notebook principal (03_Analisis_BigQuery_Demografia.ipynb)
df_nombres.to_pickle('notebooks/data/nombres_demografia.pkl')
```

In [6]:
# Función para cargar datos del análisis
def cargar_datos():
    """
    Intenta cargar los datos del análisis demográfico desde los archivos guardados
    """
    # 1. Intentar cargar desde pickle (más rápido y preserva tipos de datos)
    pkl_file = DATA_DIR / 'nombres_demografia.pkl'
    if pkl_file.exists():
        try:
            print(f"📂 Cargando datos desde pickle...")
            return pd.read_pickle(str(pkl_file))
        except Exception as e:
            print(f"⚠️ Error al cargar pickle: {str(e)}")
    
    # 2. Intentar cargar desde CSV (respaldo)
    csv_file = DATA_DIR / 'nombres_demografia.csv'
    if csv_file.exists():
        try:
            print(f"📂 Cargando datos desde CSV...")
            df = pd.read_csv(str(csv_file))
            # Asegurar que decade sea int para las visualizaciones
            if 'decade' in df.columns:
                df['decade'] = df['decade'].astype(int)
            return df
        except Exception as e:
            print(f"⚠️ Error al cargar CSV: {str(e)}")
    
    print("\n❌ No se encontraron archivos de datos en:")
    print(f"- {pkl_file}")
    print(f"- {csv_file}")
    return None

# Cargar datos
print("🔄 Iniciando carga de datos...")
df = cargar_datos()

if df is not None:
    print(f"\n✅ Datos cargados exitosamente")
    print(f"\n📊 Resumen:")
    print(f"- Registros: {len(df):,}")
    print(f"- Columnas: {', '.join(df.columns)}")
    if 'decade' in df.columns:
        print(f"- Rango temporal: {df['decade'].min()}-{df['decade'].max()}")
    
    print("\n📋 Muestra de datos:")
    display(df.head())

🔄 Iniciando carga de datos...
📂 Cargando datos desde pickle...
⚠️ Error al cargar pickle: Ran out of input
📂 Cargando datos desde CSV...

✅ Datos cargados exitosamente

📊 Resumen:
- Registros: 110
- Columnas: decade, name, gender, total_count, rank
- Rango temporal: 1910-2010

📋 Muestra de datos:


,decade,name,gender,total_count,rank
0,1910,Mary,F,478634,1
1,1910,Helen,F,248150,2
2,1910,Dorothy,F,207465,3
3,1910,Margaret,F,189228,4
4,1910,Ruth,F,173654,5


## 2. Crear Visualizaciones

Generamos las visualizaciones usando los datos guardados.

In [7]:
# Importar bibliotecas necesarias
from tqdm.notebook import tqdm
import time
import threading
from threading import Timer
import warnings

def guardar_visualizacion_segura(fig, ruta, formato='html'):
    """
    Guarda una visualización de forma segura con timeout
    
    Args:
        fig: Figura de plotly
        ruta: Ruta donde guardar
        formato: 'html' o 'png'
    
    Returns:
        bool: True si se guardó correctamente, False si falló
    """
    try:
        if formato == 'html':
            fig.write_html(
                str(ruta),
                include_plotlyjs='cdn',
                full_html=True,
                config={'displayModeBar': True}
            )
        else:  # png
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                fig.write_image(str(ruta), scale=2)
        return True
    except Exception as e:
        print(f"⚠️ Error al guardar {formato.upper()}: {str(e)}")
        return False

def crear_y_guardar_visualizacion(df=None, nombre_base='tendencias_nombres', solo_html=False):
    """
    Crea y guarda visualizaciones de tendencias de nombres con monitoreo de progreso
    
    Args:
        df: DataFrame con los datos. Si es None, intenta cargar desde archivo
        nombre_base: Nombre base para los archivos de salida
        solo_html: Si True, solo genera archivo HTML (más rápido)
    """
    try:
        start_time = time.time()
        progress = tqdm(total=100, desc="Progreso total")
        
        # 1. Cargar/verificar datos (10%)
        if df is None:
            csv_path = DATA_DIR / 'nombres_demografia.csv'
            if not csv_path.exists():
                print(f"❌ No se encontró el archivo de datos en: {csv_path}")
                return None
            print(f"📂 Cargando datos desde: {csv_path}")
            df = pd.read_csv(str(csv_path))
        progress.update(10)
        
        print(f"\n📊 Procesando {len(df):,} registros...")
        
        # 2. Preparar datos (20%)
        generos_data = {}
        for genero in ['M', 'F']:
            df_filtrado = df[df['gender'] == genero]
            nombres_populares = (df_filtrado.groupby('name')['total_count']
                            .sum()
                            .sort_values(ascending=False)
                            .head(5)
                            .index)
            generos_data[genero] = {
                'df': df_filtrado,
                'nombres': nombres_populares
            }
        progress.update(20)

        # 3. Crear figura base (10%)
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Nombres Masculinos más Populares por Década',
                        'Nombres Femeninos más Populares por Década'),
            vertical_spacing=0.12
        )
        progress.update(10)

        # 4. Añadir trazas (30%)
        print("\n📈 Generando gráficas...")
        for idx, (genero, nombre_genero) in enumerate([('M', 'Masculinos'), ('F', 'Femeninos')]):
            data = generos_data[genero]
            for nombre in tqdm(data['nombres'], desc=f"Procesando nombres {nombre_genero}"):
                datos_nombre = data['df'][data['df']['name'] == nombre]
                fig.add_trace(
                    go.Scatter(
                        x=datos_nombre['decade'],
                        y=datos_nombre['total_count'],
                        name=f"{nombre} ({nombre_genero})",
                        mode='lines+markers',
                        line=dict(width=2),
                        marker=dict(size=8)
                    ),
                    row=idx+1, col=1
                )
        progress.update(30)

        # 5. Actualizar diseño (10%)
        print("\n🎨 Actualizando diseño...")
        fig.update_layout(
            height=1000,
            width=1200,
            title_text="Evolución de Nombres más Populares por Década (1910-2013)",
            showlegend=True,
            template="plotly_white",
            font=dict(size=12)
        )
        
        for i in range(2):
            fig.update_xaxes(title_text="Década", row=i+1, col=1)
            fig.update_yaxes(title_text="Número de Nacimientos", row=i+1, col=1)
        progress.update(10)

        # 6. Guardar archivos (20%)
        print("\n💾 Guardando archivos...")
        
        # Asegurar que el directorio existe
        VIZ_DIR.mkdir(parents=True, exist_ok=True)
        
        archivos_guardados = []
        
        # Guardar HTML (siempre intentamos esto primero)
        print("Guardando HTML...", end=" ")
        ruta_html = VIZ_DIR / f'{nombre_base}.html'
        if guardar_visualizacion_segura(fig, ruta_html, 'html'):
            print(f"✅ ({ruta_html})")
            archivos_guardados.append(('HTML', ruta_html))
        
        # Guardar PNG (opcional)
        if not solo_html:
            print("Guardando PNG...", end=" ")
            ruta_png = VIZ_DIR / f'{nombre_base}.png'
            if guardar_visualizacion_segura(fig, ruta_png, 'png'):
                print(f"✅ ({ruta_png})")
                archivos_guardados.append(('PNG', ruta_png))
            else:
                print("\n⚠️ No se pudo guardar PNG. Solo se generó HTML.")
        
        progress.update(20)
        
        elapsed_time = time.time() - start_time
        print(f"\n⏱️ Tiempo total: {elapsed_time:.2f} segundos")
        
        if archivos_guardados:
            print("\n✅ Archivos generados:")
            for fmt, ruta in archivos_guardados:
                print(f"- {fmt}: {ruta}")
        
        return fig
            
    except Exception as e:
        print(f"\n❌ Error durante la generación: {str(e)}")
        return None
    finally:
        progress.close()

print("\n🚀 Iniciando generación de visualizaciones...")
print("(Este proceso puede tomar varios minutos)")

# Intentar primero solo con HTML (más rápido)
print("Intento 1: Generando solo HTML...")
fig = crear_y_guardar_visualizacion(df, solo_html=True)

if fig is not None:
    print("\n📈 Mostrando previsualización...")
    fig.show()
    
    # Preguntar si quiere intentar generar PNG
    print("\n¿Desea intentar generar también el archivo PNG? (puede tardar más)")
    print("Para generar PNG, ejecute:")
    print("fig = crear_y_guardar_visualizacion(df, solo_html=False)")
else:
    print("\n⚠️ No se pudo completar la visualización.")
    print("Sugerencia: Verifique que los datos están correctos")


🚀 Iniciando generación de visualizaciones...
(Este proceso puede tomar varios minutos)
Intento 1: Generando solo HTML...


Progreso total:   0%|          | 0/100 [00:00<?, ?it/s]


📊 Procesando 110 registros...

📈 Generando gráficas...


Procesando nombres Masculinos:   0%|          | 0/5 [00:00<?, ?it/s]

Procesando nombres Femeninos:   0%|          | 0/5 [00:00<?, ?it/s]


🎨 Actualizando diseño...

💾 Guardando archivos...
Guardando HTML... ✅ (e:\repos\ds_portfolio\notebooks\visualizaciones\tendencias_nombres.html)

⏱️ Tiempo total: 0.04 segundos

✅ Archivos generados:
- HTML: e:\repos\ds_portfolio\notebooks\visualizaciones\tendencias_nombres.html

📈 Mostrando previsualización...



¿Desea intentar generar también el archivo PNG? (puede tardar más)
Para generar PNG, ejecute:
fig = crear_y_guardar_visualizacion(df, solo_html=False)


## 3. Verificación de Resultados

Verificamos que los archivos se hayan creado correctamente.

In [9]:
def verificar_archivos():
    """
    Verifica que los archivos de visualización existan y sean accesibles
    """
    archivos = ['tendencias_nombres.png', 'tendencias_nombres.html']
    
    print("🔍 Verificando archivos generados:")
    for archivo in archivos:
        ruta = VIZ_DIR / archivo
        if ruta.exists():
            tamaño = ruta.stat().st_size / 1024  # KB
            print(f"✅ {archivo}: {tamaño:.2f} KB")
        else:
            print(f"❌ {archivo} no encontrado")

verificar_archivos()

🔍 Verificando archivos generados:
❌ tendencias_nombres.png no encontrado
✅ tendencias_nombres.html: 10.52 KB
